In [1]:
cd ..

/home/amiyaguchi/cs224w/wikipedia-retention


In [6]:
! tail -n 5 data/processed/louvain_level5_modularity.tsv

6128530 4663843
6128531 4663844
6128532 4663845
6128533 4663846
6128534 51705


In [7]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()

In [9]:
enwiki = spark.read.parquet("data/processed/enwiki-meta-compact")

In [8]:
louvain_df = spark.read.csv(
    "data/processed/louvain_level5_modularity.tsv",
    schema="user_id INT, community_id INT",
    sep=' ',
)
louvain_df.show(n=5)

+-------+------------+
|user_id|community_id|
+-------+------------+
|      0|           0|
|      1|       51705|
|      2|       51705|
|      3|       51705|
|      4|       51705|
+-------+------------+
only showing top 5 rows



In [13]:
admins = spark.read.csv("data/processed/admin_mapping.csv", schema="user_id INT, username STRING")

In [17]:
summary = (
    enwiki
    .where("cast(user_id as int) is not null")
    .withColumn("user_id", F.expr("cast(user_id as int)"))
    .groupby("user_id")
    .agg(
        F.expr("log(sum(textdata)+1) as edit_count"),
        F.expr("count(distinct article_id) as n_articles"),
        F.expr("count(*) as n_edits")
    )
    .join(admins.select("user_id", F.lit(True).alias("is_admin")), on="user_id", how="left")
    .fillna(False)
    .join(louvain_df, on="user_id", how="left")
    .fillna(-1)
)

summary.cache()
summary.count()

1525762

In [19]:
summary.createOrReplaceTempView("summary")

In [22]:
spark.sql("select distinct community_id from summary").count()

61067

In [27]:
spark.sql("""
select
    community_id,
    count(distinct user_id) as n_users,
    sum(cast(is_admin as int)) as n_admins,
    sum(edit_count),
    sum(n_edits)
from summary
group by 1
order by 2 desc
""").show()

+------------+-------+--------+------------------+------------+
|community_id|n_users|n_admins|   sum(edit_count)|sum(n_edits)|
+------------+-------+--------+------------------+------------+
|      972758| 613724|     174|5100580.8203973705|    27298829|
|     1159752| 337346|      99| 2750866.345648309|    15194061|
|       51705| 276304|     446|2277826.9965153104|    26449917|
|     1866976| 156782|      23|1341586.6485320781|     6202260|
|      313924|  55660|       6| 459204.6200409094|     2837420|
|     1604435|   6690|       1| 52006.40890295624|      141737|
|      453765|   5926|       1| 45245.67429800462|       85037|
|      190747|   2814|       0|20841.004989795914|       62041|
|       52157|   1420|       0|10773.834111095412|       18908|
|      654127|     97|       0|235.86548729764655|         102|
|     3513055|     92|       0| 724.3393082222365|         293|
|     1726367|     92|       0| 726.0197960722212|        2214|
|     2806597|     78|       0| 631.6159